In [1]:
from scipy.optimize import minimize
import numpy as np
import pandas as pd
import multiprocessing as mp
import shelve
from workers import vary_rv_worker

from helper import mse, lotka_volterra, int_cost_lotka_volterra, get_predator_prey_data

t, y = get_predator_prey_data()

In [2]:
np.random.seed(823923)
xs = np.random.uniform(low=0.5, high=10, size=(10, 4))

if False:
    shelf = shelve.open("Arrays/hc_costs")
    hc_costs = []
    hc_states = []
    
    for x0 in xs:
        res = minimize(int_cost_lotka_volterra, x0, method="nelder-mead", args=(y, t),
                       options={"xatol": 1e-8})
        x_opt = res.x
        hc_states.append(x_opt)
        cost = int_cost_lotka_volterra(x_opt, y, t)
        hc_costs.append(cost)
    
    shelf["hc_costs"] = hc_costs
    shelf["hc_states"] = hc_states
    shelf.close()

In [3]:
shelf = shelve.open("Arrays/hc_costs")
hc_costs = shelf["hc_costs"]
hc_states = shelf["hc_states"] 
shelf.close() 

In [4]:
xs

array([[0.66593614, 2.94213991, 8.129154  , 4.88824876],
       [3.13320535, 0.61170786, 1.20556467, 8.54715741],
       [1.59035578, 3.964897  , 8.93986609, 2.72514184],
       [4.73006778, 6.44811731, 5.9776969 , 7.29995916],
       [1.72004538, 8.47855451, 1.16241656, 0.58402861],
       [5.14251217, 2.59650137, 3.03574317, 7.63356504],
       [6.07600054, 4.30131415, 4.60656356, 3.67890247],
       [3.87088546, 7.40314946, 9.84267204, 5.05488566],
       [5.49825296, 0.65119381, 7.9734132 , 9.63343394],
       [7.26991767, 9.8089839 , 8.61680355, 3.66975829]])

In [6]:
hc_costs

[8.679337898561412,
 8.184897527115643,
 6.442602246343074,
 7.960944319561949,
 9.602132578994118,
 8.489120116297675,
 9.425717896170163,
 10.76325460170491,
 8.503469726735885,
 8.87407892509556]

In [7]:
def vary_rv(rvs, n_sim=50, T_start=200, T_steps=2000):
    manager = mp.Manager()
    costs = manager.list()
    states = manager.list()

    work_queue = mp.Queue()
    for i, rv in enumerate(rvs):
        costs.append(manager.list())
        states.append(manager.list())
        for _ in range(n_sim):
            work_queue.put((i, rv))

    processes = []

    for i in range(mp.cpu_count()):
        p = mp.Process(target=vary_rv_worker, args=(
            work_queue, costs, states, T_start, T_steps, t, y
        ))
        p.daemon = True
        p.start()
        processes.append(p)

    for p in processes:
        p.join()

    return [list(t) for t in costs], [list(t) for t in states]

In [8]:
%%time

if False:
    shelf = shelve.open("Arrays/sa_costs")
    np.random.seed(0o37663)
    sa_costs, sa_states = vary_rv(xs, n_sim=50, T_start=200, T_steps=2000)
    shelf["sa_costs"] = sa_costs
    shelf["sa_states"] = sa_states
    shelf.close()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs


In [26]:
shelf = shelve.open("Arrays/sa_costs")
sa_costs = np.array(shelf["sa_costs"])
sa_states = np.array(shelf["sa_states"])
shelf.close() 

In [10]:
sa_costs
print(np.mean(sa_costs, axis=1))
print(np.min(sa_costs, axis=1))
print(np.max(sa_costs, axis=1))
print(hc_costs)

[2.93546257 0.3559166  2.57610642 2.91138327 5.90656039 1.39562762
 1.47808847 1.77411622 2.60896755 1.18758241]
[0.03009545 0.02709538 0.02991884 0.05263636 0.06410285 0.02417707
 0.03065406 0.02995156 0.02824016 0.02677068]
[8.24489092 6.11662711 8.24494411 8.60605556 8.24501082 8.27674535
 8.42574474 8.43909619 8.58254819 8.75618348]
[8.679337898561412, 8.184897527115643, 6.442602246343074, 7.960944319561949, 9.602132578994118, 8.489120116297675, 9.425717896170163, 10.76325460170491, 8.503469726735885, 8.87407892509556]


In [30]:
best_sa = sa_costs.reshape(-1, sa_states.shape[-1])[np.argmin(sa_costs.flatten())]
worst_sa = sa_costs.reshape(-1, sa_states.shape[-1])[np.argmax(sa_costs.flatten())]
best_hc = hc_states[np.argmin(hc_costs)]
worst_hc = hc_states[np.argmax(hc_costs)]

IndexError: index 273 is out of bounds for axis 0 with size 125

In [22]:
best_sa, worst_sa

(0.386823138159279, 1.0265728092606916)